In [17]:
pip install pyboat


  Using cached pyboat-0.9.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached pyboat-0.9.12-py3-none-any.whl (98 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [7]:
import tifffile as tiff
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
import os, shutil
import glob
import numpy as np
from pathlib import Path
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from matplotlib.colors import to_rgba
import pyboat

Activate cellpose environment

In [2]:
# activate cellpose-env
# to use GPU, first check on the terminal: 
!nvcc --version 
!nvidia-smi
# then turn use GPU to True

from cellpose import core, utils, io, models, metrics, plot
from cellpose.io import imread
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:30:10_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
Tue Mar 25 11:23:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 5000              WDDM  |   00000000:D8:00.0 Off |                  Off |
| 33%

c:\Users\fronte\.conda\envs\cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> GPU activated? YES


Nuclei segmentation

In [10]:
# CHANGE MODEL_PATH, CHANNELS AND DIAMETER AS REQUIRED
# ADD INPUT FOLDER WHERE CROPPED TRACK IMAGES ARE

# Setup logging for Cellpose
io.logger_setup()

### MODEL
# Custom model path
# model_path = "H:/PROJECTS-03/Feyza/segmentation/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad" #membrane (w nucleus)
model_path = "H:/PROJECTS-03/Agnese/cellpose training/2d/models/nuclei_h2b" #nucleus

model = models.CellposeModel(gpu=True, pretrained_model=model_path)
### PARAMETERS
# Define channels
# channels = [[1, 2]]  # cytoplasm: 1, nucleus: 2
channels = [[0,0]] # only nucleus: if multi-channel image 3=nucleus is the third channel in the image. If there is only the nucleus (gray scale) is [0,0]

# Segmentation parameters
# diameter = 25  # in pixels for membrane
diameter = 17 # in pixels for nucleus
cellprob_threshold = -1

# Input folder containing images
input_folder = Path("E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/")
output_folder = input_folder / "denoised_segmentation_masks_nuclei"
output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

# Process all files in the folder
for img_file in input_folder.glob("*.tif"):
    print(f"Processing file: {img_file.name}")

    # Read image
    imgs = imread(img_file)
    print(f'Loaded image {img_file.name} with shape: {imgs.shape} and data type {imgs.dtype}')

    # Generate output file name
    output_file_base = output_folder / img_file.stem  # Use the base name without extension

    # USE cellposeDenoiseModel FOR DENOISING THE IMAGE
    from cellpose import denoise
    dn = denoise.DenoiseModel (model_type='denoise_cyto3', gpu=True, chan2=True)
    imgs = dn.eval(imgs, channels=[0,0], diameter=17.)

    # Perform segmentation
    masks, flows, styles = model.eval(
        imgs,
        diameter=diameter,
        channels=channels,
        cellprob_threshold=cellprob_threshold,
        do_3D=False,
        anisotropy=1.5,
        min_size=-1,
    )

    # Save output to *_seg.npy
    seg_file = str(output_file_base) + "_seg"
    io.masks_flows_to_seg(imgs, masks, flows, seg_file, channels=channels)

    # Save output masks to tiffs/pngs or txt files for ImageJ
    io.save_masks(
        imgs,
        masks,
        flows,
        str(output_file_base),
        channels=channels,
        png=False,  # Save masks as PNGs
        tif=True,  # Save masks as TIFFs
        save_txt=True,  # Save txt outlines for ImageJ
        save_flows=False,  # Save flows as TIFFs
        save_outlines=False,  # Save outlines as TIFFs
        save_mpl=False,  # Make matplotlib fig to view (WARNING: SLOW W/ LARGE IMAGES)
    )

    print(f"Saved results for {img_file.name} to {output_folder}")

print("All files processed.")

creating new log file
2025-03-25 13:46:45,536 [INFO] WRITING LOG OUTPUT TO C:\Users\fronte\.cellpose\run.log
2025-03-25 13:46:45,538 [INFO] 
cellpose version: 	3.0.10 
platform:       	win32 
python version: 	3.9.19 
torch version:  	1.8.2
2025-03-25 13:46:45,542 [INFO] ** TORCH CUDA version installed and working. **
2025-03-25 13:46:45,544 [INFO] >>>> using GPU
2025-03-25 13:46:45,743 [INFO] >>>> loading model H:/PROJECTS-03/Agnese/cellpose training/2d/models/nuclei_h2b
2025-03-25 13:46:46,009 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-03-25 13:46:46,010 [INFO] >>>> model diam_labels =  14.500 (mean diameter of training ROIs)
Processing file: Cell_2_T_0000.tif
Loaded image Cell_2_T_0000.tif with shape: (47, 45) and data type uint16
2025-03-25 13:46:46,025 [INFO] >> denoise_cyto3 << model set to be used
2025-03-25 13:46:46,027 [INFO] ** TORCH CUDA version installed and working. **
2025-03-25 13:46:46,028 [INFO] >>>> using GPU
2025-03-25 13:4

Membrane segmentation (with nucleus)

In [ ]:
# CHANGE MODEL_PATH, CHANNELS AND DIAMETER AS REQUIRED
# ADD INPUT FOLDER WHERE CROPPED TRACK IMAGES ARE

# Setup logging for Cellpose
io.logger_setup()

### MODEL
# Custom model path
model_path = "H:/PROJECTS-03/Feyza/segmentation-training/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad" #membrane (w nucleus)
#model_path = "H:/PROJECTS-03/Agnese/cellpose training/2d/models/nuclei_h2b" #nucleus

model = models.CellposeModel(gpu=True, pretrained_model=model_path)
### PARAMETERS
# Define channels
channels = [[1, 2]]  # cytoplasm: 1, nucleus: 2
#channels = [[3,0]] # only nucleus: if multi-channel image 3=nucleus is the third channel in the image. If there is only the nucleus (gray scale) is [0,0]

# Segmentation parameters
diameter = 25  # in pixels for membrane
#diameter = 17 # in pixels for nucleus
cellprob_threshold = -1

# Input folder containing images
input_folder = Path("E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/anterior_1/memnuc/")
output_folder = input_folder / "denoised_segmentation_masks_membrane"
output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

# Process all files in the folder
for img_file in input_folder.glob("*.tif"):
    print(f"Processing file: {img_file.name}")

    # Read image
    imgs = imread(img_file)
    print(f'Loaded image {img_file.name} with shape: {imgs.shape} and data type {imgs.dtype}')

    # Generate output file name
    output_file_base = output_folder / img_file.stem  # Use the base name without extension

    # USE cellposeDenoiseModel FOR DENOISING THE IMAGE
    from cellpose import denoise
    dn = denoise.DenoiseModel (model_type='denoise_cyto3', gpu=True, chan2=True)
    imgs = dn.eval(imgs, channels=[1,2], diameter=25.)

    # Perform segmentation
    masks, flows, styles = model.eval(
        imgs,
        diameter=diameter,
        channels=channels,
        cellprob_threshold=cellprob_threshold,
        do_3D=False,
        anisotropy=1.5,
        min_size=-1,
    )

    # Save output to *_seg.npy
    seg_file = str(output_file_base) + "_seg"
    io.masks_flows_to_seg(imgs, masks, flows, seg_file, channels=channels)

    # Save output masks to tiffs/pngs or txt files for ImageJ
    io.save_masks(
        imgs,
        masks,
        flows,
        str(output_file_base),
        channels=channels,
        png=False,  # Save masks as PNGs
        tif=True,  # Save masks as TIFFs
        save_txt=True,  # Save txt outlines for ImageJ
        save_flows=False,  # Save flows as TIFFs
        save_outlines=False,  # Save outlines as TIFFs
        save_mpl=False,  # Make matplotlib fig to view (WARNING: SLOW W/ LARGE IMAGES)
    )

    print(f"Saved results for {img_file.name} to {output_folder}")

print("All files processed.")

creating new log file
2025-03-21 10:48:20,252 [INFO] WRITING LOG OUTPUT TO C:\Users\fronte\.cellpose\run.log
2025-03-21 10:48:20,253 [INFO] 
cellpose version: 	3.0.10 
platform:       	win32 
python version: 	3.9.19 
torch version:  	1.8.2
2025-03-21 10:48:20,256 [WARNING] pretrained_model path does not exist, using default model
2025-03-21 10:48:20,260 [INFO] ** TORCH CUDA version installed and working. **
2025-03-21 10:48:20,261 [INFO] >>>> using GPU


2025-03-21 10:48:20,424 [INFO] >>>> loading model H:/PROJECTS-03/Feyza/segmentation-training/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad


FileNotFoundError: [Errno 2] No such file or directory: 'H:/PROJECTS-03/Feyza/segmentation-training/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad'

Isolate center nuclei and membrane

In [4]:
# function using fill holes and remove small masks util from cellpose, then picking the segment in the center of the volume and saving binarized images

# could add remove edge masks to this in the future, after making sure volumes are big enough to keep center mask away from edges

import glob

def isolate_center_mask(image, min_size=100):
    """
    Process a 2D TIFF image of masks:
    - Fill holes and remove small masks.
    - Identify the mask at the center of the center slice.
    - Keep only that mask.
    - Binarize the resulting image.
    - Save the processed image.

    Parameters:
        min_size (int): Minimum size of masks to retain during processing.
    """
    
    # Apply pre-processing: fill holes and remove small masks
    image = utils.fill_holes_and_remove_small_masks(image, min_size=min_size)
    
    # Extract the center label
    center_y, center_x = image.shape[0] // 2, image.shape[1] // 2
    central_mask_label = image[center_y, center_x]
    
    # Check if the center pixel is part of a mask
    if central_mask_label == 0:
        raise ValueError("The center of the center slice does not contain a mask.")
    
    # Create a binary mask for the entire volume
    binary_image = (image == central_mask_label).astype(np.uint8)

    return binary_image


def isolate_center_mask_in_folder(input_path, min_size=100):
    """
    Process all 3D TIFF mask files in a folder, isolating the central mask for each,
    and save the processed binary images to a new output folder.

    Parameters:
        input_path (str): Path to the folder containing mask files ending with "_masks".
        min_size (int): Minimum size of masks to retain during processing.
    """
    
    # Find all files ending with "_masks"
    mask_files = glob.glob(os.path.join(input_path, "*_masks.tif"))
    if not mask_files:
        raise ValueError("No mask files ending with '_masks.tif' were found in the input folder.")
    
    # Create an output folder in the parent directory
    parent_dir = os.path.dirname(input_path)
    output_folder = os.path.join(parent_dir, "center_masks")
    os.makedirs(output_folder, exist_ok=True)
    
    # Process each mask file
    for mask_file in mask_files:
        print(f"Processing {mask_file}...")
        # Load the mask volume
        image = tiff.imread(mask_file)
        
        # Process the volume
        try:
            binary_image = isolate_center_mask(image, min_size=min_size)
        except ValueError as e:
            print(f"Skipping {mask_file} due to error: {e}")
            continue
        
        # Save the processed binary volume
        base_name = os.path.basename(mask_file)
        output_path = os.path.join(output_folder, base_name.replace("_masks", "_center"))
        tiff.imwrite(output_path, binary_image, dtype=np.uint8)
        print(f"Saved processed file to {output_path}")
    
    print(f"Processing complete. Files saved to {output_folder}")

In [13]:
#nuclei
input_path = 'E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/denoised_segmentation_masks_nuclei/'
#membrane
#input_path1 = 'E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/anterior_1/memnuc/denoised_segmentation_masks_membrane/'

isolate_center_mask_in_folder(input_path, min_size=1)
#isolate_center_mask_in_folder(input_path1, min_size=1)

Processing E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/denoised_segmentation_masks_nuclei\Cell_2_T_0000_cp_masks.tif...
Saved processed file to E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/denoised_segmentation_masks_nuclei\center_masks\Cell_2_T_0000_cp_center.tif
Processing E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/denoised_segmentation_masks_nuclei\Cell_2_T_0001_cp_masks.tif...
Saved processed file to E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/denoised_segmentation_masks_nuclei\center_masks\Cell_2_T_0001_cp_center.tif
Processing E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/denoised_segmentation_masks_nuclei\Cell_2_T_0002_cp_masks.tif...
Saved processed file to E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/mid/nuclei/denoised_segmentation_masks_nuclei\center_masks\Cell_2_T_0002_cp_center.tif
Processing E:/Agnese/2

Tracking code

In [ ]:
#only modERK WITH MEMBRANE

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D(input_nuclear_mask_folder, input_cell_mask_folder, input_fluorescence_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    cell_mask_files = sorted([f for f in os.listdir(input_cell_mask_folder) if f.endswith(".tif")])
    fluorescence_files = sorted([f for f in os.listdir(input_fluorescence_folder) if f.endswith(".tif")])

    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    membrane_pattern = re.compile(r"Cell_(\d+)_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")

    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for cell_mask_file in cell_mask_files:
        match = membrane_pattern.match(cell_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["cell_mask"] = cell_mask_file

    for fluo_file in fluorescence_files:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if "nuclear_mask" in timepoint_dict[tp] and "cell_mask" in timepoint_dict[tp] and "fluorescence" in timepoint_dict[tp]])
        
        all_timepoints = sorted(timepoint_dict.keys())
        missing_timepoints = [tp for tp in all_timepoints if tp not in valid_timepoints]
        print(f"Missing timepoints for Cell {cell_id}: {missing_timepoints}")

        timepoints = []
        log2_ratios = []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]

            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            cell_mask = imread(os.path.join(input_cell_mask_folder, files["cell_mask"])).astype(bool)
            fluorescence = imread(os.path.join(input_fluorescence_folder, files["fluorescence"]))

            selem = disk(dilation_erosion_pixels)
            
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            eroded_cell_mask = binary_erosion(cell_mask, selem)

            cytoplasm_mask = eroded_cell_mask & ~dilated_nuclear_mask

            nucleus_intensity = fluorescence[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)

        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            
            if len(binned_log2_ratios) > 1:
                smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            else:
                smoothed_log2_ratios = binned_log2_ratios

            plt.figure(figsize=(8, 6))
            plt.plot(binned_timepoints, binned_log2_ratios, marker='o', linestyle='-', color='pink', label=f'Cell {cell_id} Binned Log2(C/N)')
            plt.xlabel("Time (minutes)")
            plt.ylabel("Log2(Cytoplasm/Nucleus Ratio)")
            plt.title(f"Cell {cell_id}: Cytoplasm/Nucleus Ratio (Binned) over Time")
            plt.grid(True)
            #plt.legend()
            plt.tight_layout()
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_log2_ratio_binned_over_time.tiff"))
            plt.close()

            plt.figure(figsize=(8, 6))
            plt.plot(binned_timepoints, smoothed_log2_ratios, marker='o', linestyle='-', color='m', label=f'Cell {cell_id} Smoothed Log2(C/N)')
            plt.xlabel("Time (minutes)")
            plt.ylabel("Smoothed Log2(Cytoplasm/Nucleus Ratio)")
            plt.title(f"Cell {cell_id}: Smoothed Cytoplasm/Nucleus Ratio (Binned) over Time")
            plt.grid(True)
            #plt.legend()
            plt.tight_layout()
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_log2_ratio_binned_smoothed.tiff"))
            plt.close()

    print("2D Processing and plotting complete!")


input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/nuclei/denoised_segmentation_masks_nuclei/center_masks/"
input_cell_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/memnuc/denoised_segmentation_masks_membrane/center_masks/"
input_fluorescence_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/erk/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/tracking_plots/"

process_and_plot_2D(input_nuclear_mask_folder, input_cell_mask_folder, input_fluorescence_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=3)

Missing timepoints for Cell 1: [0, 4, 5, 6, 7, 22, 32, 97, 101, 133]
Missing timepoints for Cell 2: [0, 1, 8, 9, 11, 12, 15, 16, 18, 34, 40, 57, 66, 76, 92, 110, 113, 122]
2D Processing and plotting complete!


In [ ]:
#with looping and with the membrane mask

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D_looping(input_nuclear_mask_folder, input_cell_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3): 
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    cell_mask_files = sorted([f for f in os.listdir(input_cell_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    membrane_pattern = re.compile(r"Cell_(\d+)_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for cell_mask_file in cell_mask_files:
        match = membrane_pattern.match(cell_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["cell_mask"] = cell_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "cell_mask", "fluorescence_1", "fluorescence_2"])] )
        
        missing_timepoints = sorted(set(timepoint_dict.keys()) - set(valid_timepoints))
        print(f"Missing timepoints for Cell {cell_id}: {missing_timepoints}")

        timepoints, log2_ratios, max_nucleus_intensities = [], [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            cell_mask = imread(os.path.join(input_cell_mask_folder, files["cell_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            eroded_cell_mask = binary_erosion(cell_mask, selem)
            cytoplasm_mask = eroded_cell_mask & ~dilated_nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]
            
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK", color='m')
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m', label='Log2(C/N) dpERK')
            ax1.tick_params(axis='y', labelcolor='m')
            
            ax2 = ax1.twinx()
            ax2.set_ylabel("looping", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue', label='looping')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            fig.tight_layout()
            #plt.legend(loc="upper left")
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_looping_mean.tiff"))
            plt.close()
    
    print("Processing and plotting complete!")

input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/nuclei/denoised_segmentation_masks_nuclei/center_masks/"
input_cell_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/memnuc/denoised_segmentation_masks_membrane/center_masks/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/posterior_1/tracking_plots/"

process_and_plot_2D_looping(input_nuclear_mask_folder, input_cell_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)

Missing timepoints for Cell 1: [0, 1, 2, 3, 4, 9, 11, 54, 65, 69]
Missing timepoints for Cell 2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 25, 27, 28, 30, 33, 34, 35, 36, 39, 48]
Missing timepoints for Cell 3: [0, 1, 2, 3, 4, 5, 8, 9, 12, 13, 14, 15, 16, 17, 20, 22, 35, 36, 37, 47, 66, 90, 94]


KeyboardInterrupt: 

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [2]:
pip install pyboat

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#with looping WITHOUT THE MEMBRANE MASK

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from pyboat import WAnalyzer, DetrendingMethods
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])] )
        #for just the first 60 timepoint
        #valid_timepoints = [tp for tp in valid_timepoints if tp*2<120]
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]
            
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK", color='m')
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m', label='Log2(C/N) dpERK')
            ax1.tick_params(axis='y', labelcolor='m')
            
            ax2 = ax1.twinx()
            ax2.set_ylabel("looping", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue', label='looping')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            fig.tight_layout()
            #plt.legend(loc="upper left")
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"mean_looping_cell_{cell_id}.tif"))
            plt.close()
    
    print("Processing and plotting complete!")


input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/posterior/nuclei/denoised_segmentation_masks_nuclei/center_masks/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/posterior/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/posterior/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/long_track/posterior/results/"

process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)


Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

In [8]:
pip install --upgrade pyboat

Note: you may need to restart the kernel to use updated packages.


In [66]:
#detrending and phase detection with pyBOAT

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.ndimage import gaussian_filter1d
from pyboat import WAnalyzer
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])] )
        #for just the first 60 timepoint
        valid_timepoints = [tp for tp in valid_timepoints if tp*2<100]
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]
            
            dt = 2 * bin_size  
            periods = np.linspace(start=2*dt, stop=48, num=200)

            # Create analyzer
            wa = WAnalyzer(periods, dt, time_unit_label='minutes')

            # Apply Sinc detrending, pyBOAT default is Sinc
            log2_detrended = wa.sinc_detrend(binned_log2_ratios, T_c=60)
            log2_detrended_smoothed = gaussian_filter1d(log2_detrended, sigma=smoothing_sigma)
            looping_detrended = wa.sinc_detrend(binned_max_nucleus_intensities, T_c=60)
            looping_detrended_smoothed = gaussian_filter1d(looping_detrended, sigma=smoothing_sigma)

            # Optional: extract phase using Hilbert transform
            log2_phase = np.angle(hilbert(log2_detrended))
            looping_phase = np.angle(hilbert(looping_detrended))

            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK activity detrended", color='plum')
            ax1.plot(binned_timepoints, log2_detrended_smoothed, marker='.', linestyle='-', color='plum', label='Detrended Log2(C/N) dpERK')
            ax1.tick_params(axis='y', labelcolor='plum')
            
            ax2 = ax1.twinx()
            ax2.set_ylabel("Her1 intensity (a.u.)", color='darkblue')
            ax2.plot(binned_timepoints, looping_detrended_smoothed, marker='.', linestyle='-', color='darkblue', label='Detrended looping')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            fig.tight_layout()
            #plt.legend(loc="upper left")
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"mean_looping_cell_{cell_id}_pink.svg"))
            plt.close()
    
    print("Processing and plotting complete!")


input_nuclear_mask_folder = "E:/Agnese/20250314_loopingxh2b_moderk_mem_late/20250314_154158_Experiment/mastodon_tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250314_loopingxh2b_moderk_mem_late/20250314_154158_Experiment/mastodon_tracking/all/erk/"
input_looping_folder = "E:/Agnese/20250314_loopingxh2b_moderk_mem_late/20250314_154158_Experiment/mastodon_tracking/all/looping/"
output_plot_folder = "E:/Agnese/20250314_loopingxh2b_moderk_mem_late/20250314_154158_Experiment/mastodon_tracking/all/detrending/"

process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=1)


Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [73]:
#detrending with also phase shift calculation in a excel file: for each peak is calculated the local phase and the phase shift between the detrended curves
#the shift is calculated considering the next looping max peak
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.signal import hilbert, find_peaks
from scipy.ndimage import gaussian_filter1d
from pyboat import WAnalyzer
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
import pandas as pd

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)
    excel_writer = pd.ExcelWriter(os.path.join(output_plot_folder, "phase_shifts.xlsx"), engine='openpyxl')

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])

    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")

    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file

    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])])
        valid_timepoints = [tp for tp in valid_timepoints if tp * 2 < 100]

        timepoints, log2_ratios, max_nucleus_intensities = [], [], []

        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask

            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]

            dt = 2 * bin_size
            periods = np.linspace(start=2 * dt, stop=48, num=200)

            wa = WAnalyzer(periods, dt, time_unit_label='minutes')
            log2_detrended = wa.sinc_detrend(binned_log2_ratios, T_c=60)
            log2_detrended_smoothed = gaussian_filter1d(log2_detrended, sigma=smoothing_sigma)
            looping_detrended = wa.sinc_detrend(binned_max_nucleus_intensities, T_c=60)
            looping_detrended_smoothed = gaussian_filter1d(looping_detrended, sigma=smoothing_sigma)

            # Hilbert phase
            log2_phase = np.angle(hilbert(log2_detrended))
            looping_phase = np.angle(hilbert(looping_detrended))

            # Peaks
            log2_peaks, _ = find_peaks(log2_detrended_smoothed)
            looping_peaks, _ = find_peaks(looping_detrended_smoothed)

            matched_peak_times = sorted(set(binned_timepoints[log2_peaks]).intersection(binned_timepoints[looping_peaks]))

            # Plot
            fig, ax1 = plt.subplots(figsize=(8, 6))
            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) modERK detrended", color='teal')
            ax1.plot(binned_timepoints, log2_detrended_smoothed, marker='.', linestyle='-', color='teal')
            ax1.tick_params(axis='y', labelcolor='teal')

            ax2 = ax1.twinx()
            ax2.set_ylabel("Her1 intensity detrended", color='darkblue')
            ax2.plot(binned_timepoints, looping_detrended_smoothed, marker='.', linestyle='-', color='darkblue')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            for pt in binned_timepoints[log2_peaks]:
                ax1.axvline(x=pt, color='teal', linestyle=':', alpha=0.5)
            for pt in binned_timepoints[looping_peaks]:
                ax2.axvline(x=pt, color='darkblue', linestyle=':', alpha=0.5)

            # Save plot
            fig.tight_layout()
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"time_{cell_id}.svg"))
            plt.close()

            # Phase diff output
            if len(log2_peaks) > 1:
                log2_period = np.median(np.diff(binned_timepoints[log2_peaks]))
            else:
                log2_period = np.nan

            # Get looping maxima only
            looping_max_peaks, _ = find_peaks(looping_detrended_smoothed)
            looping_max_times = binned_timepoints[looping_max_peaks]
            dpERK_peak_times = binned_timepoints[log2_peaks]

            # Store phase shift data
            phase_shift_data = []

            for dp_time in dpERK_peak_times:
                # Find the first looping max after the current dpERK peak
                future_maxima = looping_max_times[looping_max_times > dp_time]
                if len(future_maxima) == 0:
                    continue  # no max found after this dpERK peak

                next_max_time = future_maxima[0]
                time_delay = next_max_time - dp_time
                phase_deg = (time_delay / log2_period) * 360 if log2_period else np.nan

                phase_shift_data.append({
                    "dpERK Peak (min)": dp_time,
                    "Next Looping Max (min)": next_max_time,
                    "Time Delay (min)": time_delay,
                    "dpERK Period (min)": log2_period,
                    "Phase Shift (deg)": phase_deg
                })

            # Write to Excel (assuming you're using ExcelWriter in the outer loop)
            df_shift = pd.DataFrame(phase_shift_data)
            df_shift.to_excel(excel_writer, sheet_name=f"Cell_{cell_id}", index=False)

    excel_writer.close()
    print("Processing, plotting, and phase export complete!")


input_nuclear_mask_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/nuclei/"
input_erk_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/erk/"
input_looping_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/looping/"
output_plot_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/detrending/"

process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=1)


Processing, plotting, and phase export complete!


In [64]:
#SAVE THE CSV WITH THE RATIO VALUES! + print the same plots as before (suggestiobn: print one cell at a time and remove ID and timepoint you don't need in the analysis and it is better not to have all the cells together)
#pyboat does one cell at a time and you cannot select the id when you want two signals together

import csv
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, output_csv_path, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    # Open the CSV file for writing
    with open(output_csv_path, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        # Write the header
        writer.writerow(['Cell Number', 'Timepoint', 'Log2 Intensity (First Channel)', 'Max Nucleus Intensity (Second Channel)'])

        for cell_id, timepoint_dict in grouped_files.items():
            valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])])
            timepoints, log2_ratios, max_nucleus_intensities = [], [], []
            
            for timepoint in valid_timepoints:
                files = timepoint_dict[timepoint]
                nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
                fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
                fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))
                
                selem = disk(dilation_erosion_pixels)
                eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
                dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
                cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
                
                nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
                cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
                max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
                
                if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                    log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

            if log2_ratios:
                binned_timepoints = np.array(timepoints)[::bin_size]
                binned_log2_ratios = np.array(log2_ratios)[::bin_size]
                binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]
                
                smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
                smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

                # Save data to CSV for each timepoint
                for timepoint, log2_ratio, max_nucleus_intensity in zip(binned_timepoints, smoothed_log2_ratios, smoothed_max_nucleus_intensities):
                    writer.writerow([cell_id, timepoint, log2_ratio, max_nucleus_intensity])

                # Create the plot
                plt.figure(figsize=(8, 6))
                fig, ax1 = plt.subplots()

                ax1.set_xlabel("Time (minutes)")
                ax1.set_ylabel("Log2(C/N) dpERK", color='plum')
                ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='plum', label='Log2(C/N) dpERK')
                ax1.tick_params(axis='y', labelcolor='plum')
                
                ax2 = ax1.twinx()
                ax2.set_ylabel("looping", color='darkblue')
                ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue', label='looping')
                ax2.tick_params(axis='y', labelcolor='darkblue')

                fig.tight_layout()
                plt.grid(True)
                plt.savefig(os.path.join(output_plot_folder, f"mean_looping_cell_{cell_id}_pink.svg"))
                plt.close()

    print(f"Processing and plotting complete! CSV saved to {output_csv_path}")

# Define paths for input and output
input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/nuclei/denoised_segmentation_masks_nuclei/center_masks/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/results/"
output_csv_path = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/looping_data.csv"

process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, output_csv_path, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)


KeyboardInterrupt: 

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [34]:
#the same but detecting erk peaks and just of the first 30 timepoints

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])] )
        valid_timepoints = [tp for tp in valid_timepoints if tp*2 < 130]  # First 60 timepoints
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []

        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask

            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]

            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            # Detect peaks and minima in both channels
            peaks_1, _ = find_peaks(smoothed_log2_ratios)
            minima_1, _ = find_peaks(-smoothed_log2_ratios)  # Minima by inverting signal
            peaks_2, _ = find_peaks(smoothed_max_nucleus_intensities)
            minima_2, _ = find_peaks(-smoothed_max_nucleus_intensities)

            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK", color='m')
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m', label='Log2(C/N) dpERK')
            ax1.tick_params(axis='y', labelcolor='m')

            # Mark peaks and minima for first channel
            for p in peaks_1:
                ax1.axvline(binned_timepoints[p], color='m', linestyle="--", alpha=0.6)  # Peak line
            for m in minima_1:
                ax1.axvline(binned_timepoints[m], color='m', linestyle="--", alpha=0.6)  # Minima line

            ax2 = ax1.twinx()
            ax2.set_ylabel("Looping", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue', label='Looping')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            # Mark peaks and minima for second channel
            ax2.scatter(binned_timepoints[peaks_2], smoothed_max_nucleus_intensities[peaks_2], color='c', marker='.', edgecolors='c', s=80, zorder=3, label='Looping Peaks')
            ax2.scatter(binned_timepoints[minima_2], smoothed_max_nucleus_intensities[minima_2], color='c', marker='.', edgecolors='c', s=80, zorder=3, label='Looping Minima')

            fig.tight_layout()
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_phase_non_detrending.tif"))
            plt.close()

    print("Processing and plotting complete!")

input_nuclear_mask_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/nuclei/"
input_erk_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/erk/"
input_looping_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/looping/"
output_plot_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/detrending/"


process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=3, bin_size=1)



Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

...

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [7]:
#WITH ERK DETRENDING

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=3, bin_size=1):
    os.makedirs(output_plot_folder, exist_ok=True)

    # Get sorted files for nuclear masks and fluorescence channels
    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    # Group files by cell_id and timepoint
    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    # Process each cell and timepoint
    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])] )
        valid_timepoints = [tp for tp in valid_timepoints if tp < 40]  # First 30 timepoints
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []

        # Extract intensities for each timepoint
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            # Erosion and dilation for nuclear and cytoplasm masks
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask

            # Compute intensities for nucleus and cytoplasm
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)  # Timepoints in minutes
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

        # If there are valid log2 ratios, proceed to detrend and smooth
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]

            # 1. Apply detrending to ERK signal (log2_ratios)
            smooth_trend = gaussian_filter1d(binned_log2_ratios, sigma=15)  # Slow drift
            detrended_log2_ratios = binned_log2_ratios - smooth_trend  # Remove trend
            
            # 2. Apply smoothing to the detrended signal
            smoothed_log2_ratios = gaussian_filter1d(detrended_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            # 3. Detect peaks and minima in the detrended ERK signal
            peaks_1, _ = find_peaks(smoothed_log2_ratios, prominence=0.02)
            minima_1, _ = find_peaks(-smoothed_log2_ratios, prominence=0.02)
            peaks_2, _ = find_peaks(smoothed_max_nucleus_intensities)
            minima_2, _ = find_peaks(-smoothed_max_nucleus_intensities)

            # Plotting the results
            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            # Plot the detrended ERK signal
            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK (Detrended)", color='m')
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m', label='Detrended Log2(C/N) dpERK')
            ax1.tick_params(axis='y', labelcolor='m')

            # Mark peaks and minima for the ERK signal
            for p in peaks_1:
                ax1.axvline(binned_timepoints[p], color='m', linestyle="--", alpha=0.6)  # Peak line
            for m in minima_1:
                ax1.axvline(binned_timepoints[m], color='m', linestyle="--", alpha=0.6)  # Minima line

            # Plot the looping signal
            ax2 = ax1.twinx()
            ax2.set_ylabel("Looping", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue', label='Looping')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            # Mark peaks and minima for the looping signal
            ax2.scatter(binned_timepoints[peaks_2], smoothed_max_nucleus_intensities[peaks_2], color='c', marker='.', edgecolors='c', s=80, zorder=3, label='Looping Peaks')
            ax2.scatter(binned_timepoints[minima_2], smoothed_max_nucleus_intensities[minima_2], color='c', marker='.', edgecolors='c', s=80, zorder=3, label='Looping Minima')

            # Save the plot
            fig.tight_layout()
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_phase_detrending.tif"))
            plt.close()

    print("Processing and plotting complete!")


input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/nuclei/denoised_segmentation_masks_nuclei/center_masks/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/tailbud/results/"

process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=3, bin_size=1)


Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [33]:
#the same but with fitting with a more smoothened curve in erk
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])] )
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]
            
            # Apply the original smoothing
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')

            # Apply a stronger smoothing for the enhanced line
            more_smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=12, mode='nearest')

            smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK", color='m')

            # Original smoothed line with transparency
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m', alpha=0.3, label='Log2(C/N) dpERK (original smoothing)')

            # More smoothed line, more evident
            ax1.plot(binned_timepoints, more_smoothed_log2_ratios, marker='', linestyle='-', color='m', linewidth=2, label='Log2(C/N) dpERK (stronger smoothing)')

            ax1.tick_params(axis='y', labelcolor='m')

            ax2 = ax1.twinx()
            ax2.set_ylabel("looping", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue', label='looping')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            fig.tight_layout()
            plt.grid(True)
            plt.legend(loc="upper left")
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_smooth.tiff"))
            plt.close()
    
    print("Processing and plotting complete!")

# Paths to input and output directories
input_nuclear_mask_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/nuclei/"
input_erk_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/erk/"
input_looping_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/looping/"
output_plot_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/detrending/"

# Run the function
process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)

Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

...

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [6]:
#fitting the erk curve with a logistic (HILL) function with peak detection but without stopping the fitting at the last looping peak

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from scipy.signal import find_peaks

# Hill function (logistic function)
def hill_function(x, A, B, C, D):
    return A + (B - A) / (1 + (C / x) ** D)

def process_and_plot_2D_looping_hill_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])] )
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 3)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]
            
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            # Hill function fitting
            try:
                popt, _ = curve_fit(hill_function, binned_timepoints, smoothed_log2_ratios, maxfev=5000, p0=[min(smoothed_log2_ratios), max(smoothed_log2_ratios), np.median(binned_timepoints), 1])
                fitted_x = np.linspace(min(binned_timepoints), max(binned_timepoints), 100)
                fitted_y = hill_function(fitted_x, *popt)
            except RuntimeError:
                print(f"Curve fitting failed for Cell {cell_id}")
                popt, fitted_x, fitted_y = None, None, None

            # Peak detection in second channel (max_nucleus_intensities)
            peaks, _ = find_peaks(smoothed_max_nucleus_intensities, height=0)  # Find peaks with height > 0

            # Plotting
            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK", color='m')

            # Plot smoothed log2(C/N)
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m', alpha=0.2)

            # Plot Hill function fit
            if popt is not None:
                ax1.plot(fitted_x, fitted_y, linestyle='-', color='m')
                # Find the half-max timepoint
                max_idx = np.argmax(fitted_y) 
                max_x = fitted_x[max_idx] 
                slopes = np.diff(fitted_y) / np.diff(fitted_x)
                steep_threshold = -0.01 * (popt[1] - popt[0])
                min_decline_duration = 2
                decline_x = None
                for i in range(max_idx, len(slopes) - 1):
                    if slopes[i] < steep_threshold and all(slopes[i+j] < 0 for j in range(min_decline_duration)):
                        decline_x = fitted_x[i]
                        break
                if decline_x:
                    ax1.axvline(decline_x, color='slategrey', linestyle='-', label="Start of decline")

            ax1.tick_params(axis='y', labelcolor='m')

            ax2 = ax1.twinx()
            ax2.set_ylabel("Looping", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue')

            # Mark the detected peaks on the second channel
            ax2.plot(binned_timepoints[peaks], smoothed_max_nucleus_intensities[peaks], 'y.', label="Detected Peaks")

            ax2.tick_params(axis='y', labelcolor='darkblue')

            fig.tight_layout()
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_peaks.svg"))
            plt.close()
    
    print("Processing and plotting complete!")



input_nuclear_mask_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/nuclei/"
input_erk_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/erk/"
input_looping_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/looping/"
output_plot_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/fitting/"

process_and_plot_2D_looping_hill_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)

C:\Users\fronte\AppData\Local\Temp\21\ipykernel_49812\594235188.py:15: RuntimeWarning: divide by zero encountered in divide
  return A + (B - A) / (1 + (C / x) ** D)
C:\Users\fronte\AppData\Local\Temp\21\ipykernel_49812\594235188.py:15: RuntimeWarning: invalid value encountered in power
  return A + (B - A) / (1 + (C / x) ** D)


Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

...

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [ ]:
#plot with looping peak detection and fitting till the last peak of looping to avoid considering the erk comeback

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks, hilbert
from skimage.io import imread

def hill_function(x, A, B, C, D):
    return A + (B - A) / (1 + (C / x) ** D)

def process_and_plot_2D_looping_peak(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])

    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")

    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file

    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])] )

        timepoints, log2_ratios, max_nucleus_intensities = [], [], []

        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask

            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 3)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_max_nucleus_intensities = np.array(max_nucleus_intensities)[::bin_size]

            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(binned_max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            peaks, _ = find_peaks(smoothed_max_nucleus_intensities, prominence=0.2, distance=5)
            last_peak_time = binned_timepoints[peaks[-1]] if peaks.size > 0 else max(binned_timepoints)

            fit_mask = (binned_timepoints >= 0) & (binned_timepoints <= last_peak_time)
            x_fit = binned_timepoints[fit_mask]
            y_fit = smoothed_log2_ratios[fit_mask]

            try:
                popt, _ = curve_fit(hill_function, x_fit, y_fit, maxfev=5000, p0=[min(y_fit), max(y_fit), np.median(x_fit), 1])
                fitted_x = np.linspace(min(x_fit), max(x_fit), 100)
                fitted_y = hill_function(fitted_x, *popt)

                # Find the peak of the fitted curve
                peak_idx = np.argmax(fitted_y)
                peak_x = fitted_x[peak_idx]

            except RuntimeError:
                print(f"Curve fitting failed for Cell {cell_id}")
                popt, fitted_x, fitted_y = None, None, None
                peak_x = None

            analytic_signal = hilbert(smoothed_max_nucleus_intensities)
            instantaneous_phase = np.angle(analytic_signal)

            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()
            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK", color='m')
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m', alpha=0.2)
            if popt is not None:
                ax1.plot(fitted_x, fitted_y, linestyle='-', color='m', label="Fitted Curve")
                ax1.axvline(peak_x, color='slategrey', linestyle='-', label="Fitted Peak")
                ax1.axvline(last_peak_time, color='b', linestyle='--', alpha=0.2, label="Last Looping Peak")
            ax1.tick_params(axis='y', labelcolor='m')
            ax2 = ax1.twinx()
            ax2.set_ylabel("Looping", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_max_nucleus_intensities, marker='.', linestyle='-', color='darkblue')
            ax2.plot(binned_timepoints[peaks], smoothed_max_nucleus_intensities[peaks], 'y.', label="Peaks")
            ax2.tick_params(axis='y', labelcolor='darkblue')
            fig.tight_layout()
            plt.grid(True)
            plt.legend()
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_log2_fitted_peak.tiff"))
            plt.close()

    print("Processing and plotting complete!")


#input_nuclear_mask_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/mid/nuclei/denoised_segmentation_masks_nuclei/center_masks/"
#input_erk_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/mid/erk/"
#input_looping_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/mid/looping/"
#output_plot_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/mid/results/"
input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/mid_0/nuclei/denoised_segmentation_masks_nuclei/center_masks/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/mid_0/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/mid_0/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/mid_0/tracking_plots/"

process_and_plot_2D_looping_peak(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)


C:\Users\fronte\AppData\Local\Temp\7\ipykernel_3336\1385274518.py:23: RuntimeWarning: divide by zero encountered in divide
  return A + (B - A) / (1 + (C / x) ** D)
C:\Users\fronte\AppData\Local\Temp\7\ipykernel_3336\1385274518.py:23: RuntimeWarning: invalid value encountered in power
  return A + (B - A) / (1 + (C / x) ** D)


Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [55]:
#all the plots together colorcoded to divide them in anterior, posterior mid somite with the plots aligned to the highest peak of looping + plotting also the average
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from matplotlib.colors import to_rgba

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    # Define color mappings
    color_dict_log2 = {
        "posterior": to_rgba("magenta", 1),
        "anterior": to_rgba("darkblue", 1),
        "middle": to_rgba("orange", 1)
    }      

    color_dict_looping = {
        "posterior": to_rgba("magenta", 0.3),
        "anterior": to_rgba("darkblue", 0.3),
        "middle": to_rgba("orange", 0.3)
    }

    # Define cell ID groups
    cell_groups = {
        "posterior": [28],
        "anterior": [29],
        "middle": []
    }
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    # Collect files
    for folder, key in [(input_nuclear_mask_folder, "nuclear_mask"), 
                         (input_fluorescence_folder_1, "fluorescence_1"), 
                         (input_fluorescence_folder_2, "fluorescence_2")]:
        for file in sorted(os.listdir(folder)):
            match = nuclear_pattern.match(file) if key == "nuclear_mask" else fluorescence_pattern.match(file)
            if match:
                cell_id, timepoint = int(match.group(1)), int(match.group(2))
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})[key] = file

    plt.figure(figsize=(10, 6))
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    aligned_data = []

    for cell_id, timepoint_dict in grouped_files.items():
        group = next((key for key, ids in cell_groups.items() if cell_id in ids), None)
        if not group:
            continue  # Skip cells not in the defined groups
        
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(k in timepoint_dict[tp] for k in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])])
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []

        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask

            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

        if log2_ratios:
            smoothed_log2_ratios = gaussian_filter1d(log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            # Find peak of looping signal (max value)
            peak_idx = np.argmax(smoothed_max_nucleus_intensities)
            peak_time = timepoints[peak_idx]

            # Align both signals by shifting the time axis
            aligned_timepoints = np.array(timepoints) - peak_time
            
            aligned_data.append((aligned_timepoints, smoothed_log2_ratios, smoothed_max_nucleus_intensities, group))

    # Plot the aligned data
    for timepoints, smoothed_log2_ratios, smoothed_max_nucleus_intensities, group in aligned_data:
        ax1.plot(timepoints, smoothed_log2_ratios, marker='', linestyle='-', color=color_dict_log2[group])
        ax2.plot(timepoints, smoothed_max_nucleus_intensities, marker='', linestyle='--', color=color_dict_looping[group])

    # Compute mean across groups
    all_times = np.unique(np.concatenate([d[0] for d in aligned_data]))
    mean_log2_ratios = {group: [] for group in cell_groups}
    mean_looping = {group: [] for group in cell_groups}

    for group in cell_groups:
        group_data = [d for d in aligned_data if d[3] == group]
        if group_data:
            interpolated_log2_ratios = np.array([np.interp(all_times, d[0], d[1]) for d in group_data])
            interpolated_looping = np.array([np.interp(all_times, d[0], d[2]) for d in group_data])
            
            mean_log2_ratios[group] = np.mean(interpolated_log2_ratios, axis=0)
            mean_looping[group] = np.mean(interpolated_looping, axis=0)

            ax1.plot(all_times, mean_log2_ratios[group], color=color_dict_log2[group], label=f'{group}', linewidth=2)
            ax2.plot(all_times, mean_looping[group], color=color_dict_looping[group], label=f'{group}', linewidth=2)

    ax1.set_xlabel("Aligned Time (minutes)")
    ax1.set_ylabel("Log2(C/N) dpERK", color='black')
    ax2.set_ylabel("Looping", color='black')
    ax1.legend()

    fig.tight_layout()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "neighs_s-1.svg"))
    plt.close()

    print("Processing and aligned plotting complete!")


input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/"


process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)

Processing and aligned plotting complete!


<Figure size 3000x1800 with 0 Axes>

In [35]:
#again aligning with the last peak but plotting mean and standard deviation after alignment

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from matplotlib.colors import to_rgba

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    # Define color mappings
    color_dict_log2 = {
        "posterior": to_rgba("magenta", 1),
        "anterior": to_rgba("darkblue", 1),
        "middle": to_rgba("orange", 1)
    }      

    color_dict_looping = {
        "posterior": to_rgba("magenta", 1),
        "anterior": to_rgba("darkblue", 1),
        "middle": to_rgba("orange", 1)
    }

    # Define cell ID groups
    cell_groups = {
        "posterior": [1,2,3,4],
        "anterior": [7,8,9],
        "middle": []
    }
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    # Collect files
    for folder, key in [(input_nuclear_mask_folder, "nuclear_mask"), 
                         (input_fluorescence_folder_1, "fluorescence_1"), 
                         (input_fluorescence_folder_2, "fluorescence_2")]:
        for file in sorted(os.listdir(folder)):
            match = nuclear_pattern.match(file) if key == "nuclear_mask" else fluorescence_pattern.match(file)
            if match:
                cell_id, timepoint = int(match.group(1)), int(match.group(2))
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})[key] = file

    plt.figure(figsize=(10, 6))
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    aligned_data = []

    for cell_id, timepoint_dict in grouped_files.items():
        group = next((key for key, ids in cell_groups.items() if cell_id in ids), None)
        if not group:
            continue  # Skip cells not in the defined groups
        
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(k in timepoint_dict[tp] for k in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])])
        #for just the first 60 timepoint
        #valid_timepoints = [tp for tp in valid_timepoints if tp*2<60]
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []

        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask

            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

        if log2_ratios:
            smoothed_log2_ratios = gaussian_filter1d(log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            # Find peak of looping signal (max value)
            peak_idx = np.argmax(smoothed_max_nucleus_intensities)
            peak_time = timepoints[peak_idx]

            # Align both signals by shifting the time axis
            aligned_timepoints = np.array(timepoints) - peak_time
            
            aligned_data.append((aligned_timepoints, smoothed_log2_ratios, smoothed_max_nucleus_intensities, group))

    # Compute mean and standard deviation across groups
    all_times = np.unique(np.concatenate([d[0] for d in aligned_data]))
    mean_log2_ratios = {group: [] for group in cell_groups}
    std_log2_ratios = {group: [] for group in cell_groups}
    mean_looping = {group: [] for group in cell_groups}
    std_looping = {group: [] for group in cell_groups}

    for group in cell_groups:
        group_data = [d for d in aligned_data if d[3] == group]
        if group_data:
            interpolated_log2_ratios = np.array([np.interp(all_times, d[0], d[1]) for d in group_data])
            interpolated_looping = np.array([np.interp(all_times, d[0], d[2]) for d in group_data])
            
            mean_log2_ratios[group] = np.mean(interpolated_log2_ratios, axis=0)
            std_log2_ratios[group] = np.std(interpolated_log2_ratios, axis=0)
            
            mean_looping[group] = np.mean(interpolated_looping, axis=0)
            std_looping[group] = np.std(interpolated_looping, axis=0)

            # Plotting the mean and standard deviation
            ax1.plot(all_times, mean_log2_ratios[group], color=color_dict_log2[group], label=f'{group} Mean', linewidth=2)
            ax2.plot(all_times, mean_looping[group], color=color_dict_looping[group], label=f'{group} Mean', linewidth=2, linestyle= '--')

            # Fill the area between the mean ± std
            ax1.fill_between(all_times, mean_log2_ratios[group] - std_log2_ratios[group], mean_log2_ratios[group] + std_log2_ratios[group], color=color_dict_log2[group], alpha=0.1)
            #ax2.fill_between(all_times, mean_looping[group] - std_looping[group], mean_looping[group] + std_looping[group], color=color_dict_looping[group], alpha=0.2)

    ax1.set_xlabel("Aligned Time (minutes)")
    ax1.set_ylabel("Log2(C/N) dpERK", color='black')
    ax2.set_ylabel("Looping", color='black')
    ax1.legend()
    ax2.legend()
    fig.tight_layout()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "S0_S1_neigh_mean.svg"))
    plt.close()

    print("Processing, alignment, and plotting with mean and standard deviation complete!")


input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/"

#input_nuclear_mask_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/nuclei/"
#input_erk_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/erk/"
#input_looping_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/looping/"
#output_plot_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/results/"

#input_nuclear_mask_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/all/nuclei/"
#input_erk_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/all/erk/"
#input_looping_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/all/looping/"
#output_plot_folder = "H:/PROJECTS-03/Agnese/20241107_loopingxh2b_inj_modERK_mem/20241107_182400_Experiment-18_20ss/mastodon_tracking/all/results/"

process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)


Processing, alignment, and plotting with mean and standard deviation complete!


<Figure size 3000x1800 with 0 Axes>

In [ ]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from matplotlib.colors import to_rgba

def process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    # Define color mappings
    color_dict_log2 = {
        "posterior": to_rgba("magenta", 1),
        "anterior": to_rgba("darkblue", 1),
        "middle": to_rgba("orange", 1)
    }      

    color_dict_looping = {
        "posterior": to_rgba("magenta", 1),
        "anterior": to_rgba("darkblue", 1),
        "middle": to_rgba("orange", 1)
    }

    # Define cell ID groups
    cell_groups = {
        "posterior": [26,27,28],
        "anterior": [29,30,31,32],
        "middle": []
    }
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    # Collect files
    for folder, key in [(input_nuclear_mask_folder, "nuclear_mask"), 
                         (input_fluorescence_folder_1, "fluorescence_1"), 
                         (input_fluorescence_folder_2, "fluorescence_2")]:
        for file in sorted(os.listdir(folder)):
            match = nuclear_pattern.match(file) if key == "nuclear_mask" else fluorescence_pattern.match(file)
            if match:
                cell_id, timepoint = int(match.group(1)), int(match.group(2))
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})[key] = file

    plt.figure(figsize=(10, 6))
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    aligned_data = []

    for cell_id, timepoint_dict in grouped_files.items():
        group = next((key for key, ids in cell_groups.items() if cell_id in ids), None)
        if not group:
            continue  # Skip cells not in the defined groups
        
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(k in timepoint_dict[tp] for k in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])])
        # Consider only the first 60 timepoints
        #valid_timepoints = [tp for tp in valid_timepoints if tp*2 < 120]
        
        timepoints, log2_ratios, max_nucleus_intensities = [], [], []

        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask

            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            max_nucleus_intensity = fluorescence_2[eroded_nuclear_mask].max() if np.any(eroded_nuclear_mask) else 0

            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    max_nucleus_intensities.append(max_nucleus_intensity)

        if log2_ratios:
            smoothed_log2_ratios = gaussian_filter1d(log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_max_nucleus_intensities = gaussian_filter1d(max_nucleus_intensities, sigma=smoothing_sigma, mode='nearest')

            # Find the last peak of looping signal
            peaks, _ = find_peaks(smoothed_max_nucleus_intensities)
            if len(peaks) > 0:
                peak_idx = peaks[-1]  # Last peak
                peak_time = timepoints[peak_idx]

                # Align both signals by shifting the time axis
                aligned_timepoints = np.array(timepoints) - peak_time
                aligned_data.append((aligned_timepoints, smoothed_log2_ratios, smoothed_max_nucleus_intensities, group))

    # Compute mean and standard deviation across groups
    all_times = np.unique(np.concatenate([d[0] for d in aligned_data]))
    mean_log2_ratios = {group: [] for group in cell_groups}
    std_log2_ratios = {group: [] for group in cell_groups}
    mean_looping = {group: [] for group in cell_groups}
    std_looping = {group: [] for group in cell_groups}

    for group in cell_groups:
        group_data = [d for d in aligned_data if d[3] == group]
        if group_data:
            interpolated_log2_ratios = np.array([np.interp(all_times, d[0], d[1]) for d in group_data])
            interpolated_looping = np.array([np.interp(all_times, d[0], d[2]) for d in group_data])
            
            mean_log2_ratios[group] = np.mean(interpolated_log2_ratios, axis=0)
            std_log2_ratios[group] = np.std(interpolated_log2_ratios, axis=0)
            
            mean_looping[group] = np.mean(interpolated_looping, axis=0)
            std_looping[group] = np.std(interpolated_looping, axis=0)

            # Plotting the mean and standard deviation
            ax1.plot(all_times, mean_log2_ratios[group], color=color_dict_log2[group], label=f'{group} Mean', linewidth=2)
            ax2.plot(all_times, mean_looping[group], color=color_dict_looping[group], label=f'{group} Mean', linewidth=2, linestyle='--')

            # Fill the area between the mean ± std
            ax1.fill_between(all_times, mean_log2_ratios[group] - std_log2_ratios[group], mean_log2_ratios[group] + std_log2_ratios[group], color=color_dict_log2[group], alpha=0.1)
            ax2.fill_between(all_times, mean_looping[group] - std_looping[group], mean_looping[group] + std_looping[group], color=color_dict_looping[group], alpha=0.1)

    ax1.set_xlabel("Aligned Time (minutes)")
    ax1.set_ylabel("Log2(C/N) dpERK", color='black')
    ax2.set_ylabel("Looping", color='black')
    ax1.legend()
    ax2.legend()
    fig.tight_layout()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "postantsucc.tiff"))
    plt.close()

    print("Processing, alignment (by last peak), and plotting with mean and standard deviation complete!")

input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/erk/"
input_looping_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/looping/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/tracking_plot/"

# Call the function with your file paths
process_and_plot_2D_looping_nuc(input_nuclear_mask_folder, input_erk_folder, input_looping_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)

Processing, alignment (by last peak), and plotting with mean and standard deviation complete!


<Figure size 3000x1800 with 0 Axes>